In [74]:
import numpy as np
import pandas as pd
from scipy.stats import gmean
RI = np.array([0,0,0.58,0.90,1.12,1.24,1.32,1.41,1.45,1.49,1.51,1.48,1.56,1.57,1.58])
RIm = np.array([0,0,0.4889,0.7937,1.072,1.1996,1.2874,1.3410,1.3793,1.4095,1.4181,1.4462,1.4555,1.4913,1.4986])
RIg = np.array([1,2,0.1796,0.2627,0.3597,0.3818,0.4090,0.4164,0.4348,0.4455,0.4536,0.4776,0.4691,0.4804,0.4880])

#subcriteria
df =pd.read_csv("WesternGhatData/1Elevation.csv")
elevation= df.to_numpy()

df =pd.read_csv("WesternGhatData/2Slope.csv")
slope= df.to_numpy()

df =pd.read_csv("WesternGhatData/3DistancefromRiver.csv")
river= df.to_numpy()

df =pd.read_csv("WesternGhatData/4Drainage.csv")
drainage= df.to_numpy()

df =pd.read_csv("WesternGhatData/5FlowAccumulation.csv")
flow= df.to_numpy()

df =pd.read_csv("WesternGhatData/6TWI.csv")
twi= df.to_numpy()

df =pd.read_csv("WesternGhatData/7Rainfall.csv")
rainfall= df.to_numpy()

df =pd.read_csv("WesternGhatData/8Landuse.csv")
landuse= df.to_numpy()

df =pd.read_csv("WesternGhatData/9SoilTexture.csv")
soil= df.to_numpy()

df =pd.read_csv("WesternGhatData/10TRI.csv")
tri= df.to_numpy()

df =pd.read_csv("WesternGhatData/11Geology.csv")
geology= df.to_numpy()

data = np.array([[1   ,2   ,2   ,3   ,4   ,5   ,5   ,6   ,7   ,8   ,8],
                   [1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,6   ,7   ,8   ,8],
                   [1/2 ,1/2 ,1   ,2   ,3   ,4   ,4   ,5   ,6   ,7   ,7],
                   [1/3 ,1/3 ,1/2 ,1   ,2   ,3   ,3   ,4   ,5   ,7   ,8],
                   [1/4 ,1/3 ,1/3 ,1/2 ,1   ,2   ,2   ,3   ,4   ,5   ,6],
                   [1/5 ,1/4 ,1/4 ,1/3 ,1/2 ,1   ,2   ,4   ,5   ,7   ,7],
                   [1/5 ,1/5 ,1/4 ,1/3 ,1/2 ,1/2 ,1   ,2   ,3   ,5   ,6],
                   [1/6 ,1/6 ,1/5 ,1/4 ,1/3 ,1/4 ,1/2 ,1   ,2   ,4   ,5],
                   [1/7 ,1/7 ,1/6 ,1/5 ,1/4 ,1/5 ,1/3 ,1/2 ,1   ,3   ,4],
                   [1/8 ,1/8 ,1/7 ,1/7 ,1/5 ,1/7 ,1/5 ,1/4 ,1/3 ,1   ,3],
                   [1/8 ,1/8 ,1/7 ,1/8 ,1/6 ,1/7 ,1/6 ,1/5 ,1/4 ,1/3 ,1]]
                  )

n = data[0].size
float_formatter = "{:.3f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

In [ ]:
mainarr = np.full((n,n,3),0,dtype=float)
for i in range(0,n):
    for j in range(0,n):
        if (data[i][j]>=1):
            mainarr[i][j][0]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]-1
            mainarr[i][j][2]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]+1
        else:
            for k in range(2,10):
                if (1/k==data[i][j]):
                    mainarr[i][j][0]= data[i][j] if(data[i][j]==9) else 1/(k+1)
                    mainarr[i][j][2]= data[i][j] if(data[i][j]==9) else 1/(k-1)
        mainarr[i][j][1]= data[i][j]
print(mainarr)

In [138]:
#converts normal into fuzzyarray
def fuzzify(data):
    n = data[0].size
    mainarr = np.full((n,n,3),0,dtype=float)
    # fuzzedGM = np.full((n,n),0,dtype=float)
    for i in range(0,n):
        for j in range(0,n):
            if (data[i][j]>=1):
                mainarr[i][j][0]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]-1
                mainarr[i][j][2]= data[i][j] if(data[i][j]==1 or data[i][j]==9) else data[i][j]+1
            else:
                for k in range(2,10):
                    if (1/k==data[i][j]):
                        mainarr[i][j][0]= data[i][j] if(data[i][j]==9) else 1/(k+1)
                        mainarr[i][j][2]= data[i][j] if(data[i][j]==9) else 1/(k-1)
            mainarr[i][j][1]= data[i][j]
            # fuzzedGM[i][j]=gmean([mainarr[i][j][0],mainarr[i][j][2]])
    # print(fuzzedGM)
    # return mainarr,fuzzedGM
    return mainarr

In [139]:
def criteriaWeights(mainarr):
    n = mainarr[0].size
    colsum = mainarr.sum(axis=0)
    # normalized form 
    i=0 
    normalizedform = np.array(mainarr)
    for p in mainarr:
        normalizedform[i] = np.divide(mainarr[i] , colsum)
        i+=1
    criteriaweights = normalizedform.sum(axis=1)/n
    return criteriaweights

def FuzzycriteriaWeights(mainarr):
    n = mainarr[0].size if (np.ndim(mainarr)==2) else int(mainarr[0].size/3)
    fuzzyGM = np.full((n,3),1,dtype=float)
    for i in range(0,n):
        for j in range(0,n):
            fuzzyGM[i] = np.multiply(fuzzyGM[i],mainarr[i][j])
        fuzzyGM[i]=np.power(fuzzyGM[i],1/n)
    fuzzyWeights=np.full((n,3),1,dtype=float)
    for i in range(0,n):
        fuzzyWeights[i]=np.multiply(fuzzyGM[i],np.reciprocal(np.flip(fuzzyGM.sum(axis=0),0)))
    weights = np.average(fuzzyWeights,axis=1)
    weights = np.divide(weights,weights.sum(axis=0))
    return weights

In [142]:
def consistency(mainarr):
    n = int(mainarr[0].size)
    print(n)

    # fuzzyArr,fuzzedGM = fuzzify(mainarr)
    fuzzyArr = fuzzify(mainarr)
    
    def consIndex(mainarr):
        n = int(mainarr[0].size)
        c1 = np.array(mainarr)
        
        # print("naipata",criteriaWeights(mainarr))
        for i in range(0,n):
            c1[i] = np.multiply(mainarr[i],criteriaWeights((mainarr)))
        weightedsum = c1.sum(axis=1)
        naipata = np.divide(weightedsum,criteriaWeights((mainarr)))
        lamda = naipata.sum(axis=0)/n
        # print("l",lamda)
        return (lamda-n)/(n-1)


    CR = consIndex(mainarr) / RI[n-1]
    print("cr ",CR)

    # CR2 = consIndex(mainarr) / RIm[n-1]
    # print("cr2",CR2)

    # CR3 = consIndex(fuzzedGM) / RIg[n-1]
    # print("CR3",CR3)and CR2<0.1 and CR3<0.1

    if(CR<0.10 ):
        print("VALUES ARE CONSISTENT - go ahead")
        print("cwAHP",criteriaWeights(mainarr))
        # print("cwfuzzedgm",criteriaWeights(fuzzedGM))
        print("cwfuzzyAHP",FuzzycriteriaWeights(fuzzyArr))
        # return criteriaWeights((mainarr))
    else:
        print("criteria weights are not consistent reEnter the values")
        print("cwmain",criteriaWeights(mainarr))
        # print("cwfuzzedgm",criteriaWeights(fuzzedGM))
        print("cwfuzzyArr",criteriaWeights(fuzzyArr))
        # return criteriaWeights((mainarr))


In [143]:
print(consistency(data))
# print("a",criteriaWeights(elevation))
# print("a",FuzzycriteriaWeights(elevation))

11
cr  0.06480286359154262
VALUES ARE CONSISTENT - go ahead
cwAHP [0.236 0.195 0.152 0.112 0.078 0.075 0.053 0.038 0.028 0.019 0.014]
cwfuzzyAHP [0.231 0.198 0.160 0.117 0.082 0.070 0.053 0.036 0.026 0.016 0.012]
None


In [144]:
print(consistency(elevation),"elevation\n\n")
print(consistency(slope),"slope\n\n")
print(consistency(river),"river\n\n")
print(consistency(drainage),"drainage\n\n")
print(consistency(flow),"flow\n\n")
print(consistency(twi),"twi\n\n")
print(consistency(rainfall),"rainfall\n\n")
print(consistency(landuse),"landuse\n\n")
print(consistency(soil),"soil\n\n")
print(consistency(tri),"tri\n\n")
print(consistency(geology),"geology\n\n")

5
cr  0.02159551613867609
VALUES ARE CONSISTENT - go ahead
cwAHP [0.518 0.218 0.139 0.084 0.041]
cwfuzzyAHP [0.694 0.070 0.197 0.026 0.013]
None elevation


5
cr  0.029474590206579408
VALUES ARE CONSISTENT - go ahead
cwAHP [0.499 0.276 0.113 0.070 0.043]
cwfuzzyAHP [0.840 0.088 0.036 0.022 0.014]
None slope


5
cr  0.04215341953280287
VALUES ARE CONSISTENT - go ahead
cwAHP [0.400 0.258 0.197 0.102 0.043]
cwfuzzyAHP [0.534 0.367 0.057 0.029 0.013]
None river


5
cr  0.013463356735784034
VALUES ARE CONSISTENT - go ahead
cwAHP [0.433 0.262 0.164 0.089 0.052]
cwfuzzyAHP [0.553 0.356 0.049 0.027 0.015]
None drainage


5
cr  0.011394546959168321
VALUES ARE CONSISTENT - go ahead
cwAHP [0.423 0.287 0.151 0.088 0.050]
cwfuzzyAHP [0.486 0.345 0.046 0.108 0.015]
None flow


5
cr  0.04175341930908182
VALUES ARE CONSISTENT - go ahead
cwAHP [0.421 0.253 0.192 0.090 0.045]
cwfuzzyAHP [0.551 0.354 0.056 0.026 0.013]
None twi


5
cr  0.015238959298032435
VALUES ARE CONSISTENT - go ahead
cwAHP [0.416 0.

In [7]:
#function made
def bene(arr):
    num=0
    while num<n:
        ite=0
        max=np.max(arr[num])
        min=np.min(arr[num])
        for i in arr[num] :
            if(num != 0):
                arr[num][ite]=i/max
            else:
                arr[num][ite]=min/i
            ite+=1
        num+=1

In [ ]:
#Tla transpose list array   ctla copy of tla
listarray = np.array([[250,16,12,5],[200,16,8,3],[300,32,16,4],[275,32,8,4],[225,16,16,2]])
print("\n")
Tla = listarray.T
Tla = Tla.astype('float64') 

# function called
bene(Tla)
Tla = Tla.T
print(Tla)



In [ ]:
cw = 0
for i in Tla:
    Tla[cw] = np.multiply(Tla[cw],criteriaweights)
    cw+=1
print(Tla)


In [ ]:
#weighted normalized decision matrix

# out_arr = np.multiply(in_arr1, in_arr2) ye galat hai for what i am doin     

# out_arr=np.array(in_arr1)*np.array(in_arr2) this is right

# wndm = np.array(Tla)*np.array(cw)
# print(wndm)
#now this is according to the other video example


In [ ]:
perfscore = Tla.sum(axis=1)
print (perfscore)

In [ ]:
rankArr = np.array(perfscore)
ranks = perfscore.size
for score in rankArr :
    rankArr[rankArr.argmin()]=ranks
    ranks-=1
print(rankArr)

In [ ]:
finalarr = np.column_stack([listarray,rankArr])
print(finalarr)

In [ ]:
finalarr = finalarr[finalarr[:,finalarr[0].size-1].argsort()]
print(finalarr)